# Notes

This architecture was described in "Deep learning with convolutional neural networks for brain mapping and decoding of movement-related information from the human EEG", by R. T. Schmirrmester et al, 2018. In this notebook we conduct experimetns showing the importance of using batchnorm/dropout layers, and dependacy between accuracy and the number of timestamps in a sample.

# Set up the environment

In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# import tf
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K

# import os functions
import os
import time

import numpy as np
import matplotlib.pyplot as plt

from IPython import display

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
cd 'drive/My Drive/147 Project'

[Errno 2] No such file or directory: 'drive/My Drive/147 Project'
/content/drive/My Drive/147 Project


# Load the data

### Read the files

In [0]:
X_test = np.load("./EEG_data/X_test.npy")
y_test = np.load("./EEG_data/y_test.npy") - 769
person_train_valid = np.load("./EEG_data/person_train_valid.npy")
X_train_valid = np.load("./EEG_data/X_train_valid.npy")
y_train_valid = np.load("./EEG_data/y_train_valid.npy") - 769
person_test = np.load("./EEG_data/person_test.npy")

### Shape of data

In [24]:
print("training/Valid data shape: {}".format(X_train_valid.shape))       # training data of many persons
print("Test data shape: {}".format(X_test.shape))                        # test data of many persons
print("Training/Valid target shape: {}".format(y_train_valid.shape))     # training labels of many persons
print("Test target shape: {}".format(y_test.shape))                      # test labels of many persons
print("Person train/valid  shape: {}".format(person_train_valid.shape))  # which person correspond to the trail in test set
print("Person test shape: {}".format(person_test.shape))                 # which person correspond to the trail in test set

training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid  shape: (2115, 1)
Person test shape: (443, 1)


### divide dataset into training and validation

In [25]:
perm = np.random.permutation(X_train_valid.shape[0])
num_train = int(0.8 * X_train_valid.shape[0])
num_valid = X_train_valid.shape[0] - num_train
X_train =  X_train_valid[perm[0:num_train]]
y_train =  y_train_valid[perm[0:num_train]]
X_valid = X_train_valid[perm[num_train: ]]
y_valid = y_train_valid[perm[num_train: ]]


print("Training data shape: {}".format(X_train.shape))
print("Training label shape: {}".format(y_train.shape))
print("Validation data shape: {}".format(X_valid.shape))
print("Validation label shape: {}".format(y_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Test label shape: {}".format(y_test.shape))

Training data shape: (1692, 22, 1000)
Training label shape: (1692,)
Validation data shape: (423, 22, 1000)
Validation label shape: (423,)
Test data shape: (443, 22, 1000)
Test label shape: (443,)


### Augmented dataset

In [0]:
def sliding_window(X_arr, y_arr, time_window=100, time_step=1, time_stride=1):
    temp_x = np.moveaxis(X_arr, 2, 0)
    temp_x = temp_x.astype(np.float32)
    buff = []
    
    num_slices = (len(temp_x)-time_window*time_step) // time_stride + 1
    
    # get time slices for data
    for i in range(num_slices):
        buff.append(temp_x[i*time_stride:i*time_stride + time_window*time_step:time_step])
        buff[i] = np.moveaxis(buff[i], 0, 2)
        # uncomment this if additional dimension is needed
        # buff[i] = buff[i].reshape(1, buff[i].shape[0], buff[i].shape[1], buff[i].shape[2])
        
    temp_x = np.concatenate(buff)
        
    # get time slice for labels
    temp_y = np.ones((X_arr.shape[0],num_slices))
    
    for i in range(len(y_arr)):
        temp_y[i] = temp_y[i] * y_arr[i]
        
    temp_y = temp_y.reshape((-1))
    
    return temp_x, temp_y

In [0]:
def construct_augmented_deep_model(TIME_WINDOW):
    # input
    deep_aug_input = layers.Input(shape=(22, TIME_WINDOW))


    # ================================== CONV1 ================================== #

    # conv accross time domain
    r1 = layers.Reshape((22, TIME_WINDOW, 1))(deep_aug_input)
    c1 = layers.Conv2D(25, (1, 10), strides=(1, 1))(r1)
    new_size = TIME_WINDOW - 10 + 1
    t1 = tf.keras.layers.Permute((2, 3, 1))(c1)

    # # conv accross channels
    r2 = layers.Reshape((new_size, 25*22, 1))(t1)
    c2 = layers.Conv2D(25, (1, 25*22), strides=(1, 1))(r2)
    bn2 = layers.BatchNormalization(axis=1)(c2)                 # do I use the right filter?
    a2 = layers.Activation("elu")(bn2)


    # max pool across time domain
    r3 = layers.Reshape((new_size, 25, 1))(a2)
    maxpool3 = layers.MaxPooling2D(pool_size=(3, 1), strides=(3, 1))(r3)
    new_size = (new_size - 3)//3 + 1
    
    # # =========================================================================== #



    # # ================================= CONV2-4 ================================= #

    c4 = layers.Conv2D(50, (10, 25), strides=(1, 1))(maxpool3)
    new_size = new_size - 10 + 1
    bn4 = layers.BatchNormalization(axis=1)(c4)
    a4 = layers.Activation("elu")(bn4)
    do4 = layers.Dropout(0.5)(a4)
    r4 = layers.Reshape((new_size, 50, 1))(do4)
    maxpool4 = layers.MaxPooling2D(pool_size=(3, 1), strides=(3, 1))(r4)
    new_size = (new_size - 3)//3 + 1
    

    c5 = layers.Conv2D(100, (10, 50), strides=(1, 1))(maxpool4)
    new_size = new_size - 10 + 1
    bn5 = layers.BatchNormalization(axis=1)(c5)
    a5 = layers.Activation("elu")(bn5)
    do5 = layers.Dropout(0.5)(a5)
    r5 = layers.Reshape((new_size, 100, 1))(do5)
    maxpool5 = layers.MaxPooling2D(pool_size=(3, 1), strides=(3, 1))(r5)
    new_size = (new_size - 3)//3 + 1
    

    c6 = layers.Conv2D(200, (10, 100), strides=(1, 1))(maxpool5)
    new_size = new_size - 10 + 1
    bn6 = layers.BatchNormalization(axis=1)(c6)
    a6 = layers.Activation("elu")(bn6)
    do6 = layers.Dropout(0.5)(a6)
    r6 = layers.Reshape((new_size, 200, 1))(do6)
    maxpool6 = layers.MaxPooling2D(pool_size=(3, 1), strides=(3, 1))(r6)

    # # =========================================================================== #

    f7 = layers.Flatten()(do6)

    # output
    deep_aug_output = layers.Dense(4, activation="softmax")(f7)
    
    return keras.Model(inputs = deep_aug_input, outputs = deep_aug_output, name="deep_aug_model")

### Train on Single Person dataset

In [28]:
person_num = 0
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))

deep_aug_model_600_005 = construct_augmented_deep_model(TIME_WINDOW)
deep_aug_model_600_005.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

deep_aug_model_600_005.fit(X_train_slices, y_train_slices,
                       validation_data = (X_valid_slices, y_valid_slices),
                       epochs = 60)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

deep_aug_model_600_005.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

deep_aug_model_600_005.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (189, 22, 1000)
Training label shape for 1 person: (189,)
Validation data shape for 1 person: (48, 22, 1000)
Validation label shape for 1 person: (48,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)
Training data shape with slices: (189, 22, 600)
Training label shape with slice: (189,)
Validation data shape with slices: (48, 22, 600)
Validation label shape with slice: (48,)
Train on 189 samples, validate on 48 samples
Epoch 1/60
189/189 [==============================] - 1s 4ms/sample - loss: 3.1019 - acc: 0.3175 - val_loss: 1.6097 - val_acc: 0.1875
Epoch 2/60
189/189 [==============================] - 0s 374us/sample - loss: 2.1036 - acc: 0.2540 - val_loss: 1.5882 - val_acc: 0.2917
Epoch 3/60
189/189 [==============================] - 0s 383us/sample - loss: 1.9759 - acc: 0.2381 - val_loss: 1.5692 - val_acc: 0.2708
Epoch 4/60
189/189 [==============================] - 0s 377us/sample - loss: 1.8695 - acc: 0.3122 - va

[2.4027202764429303, 0.31376976]

In [29]:
person_num = 1
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))

deep_aug_model_600_005 = construct_augmented_deep_model(TIME_WINDOW)
deep_aug_model_600_005.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

deep_aug_model_600_005.fit(X_train_slices, y_train_slices,
                       validation_data = (X_valid_slices, y_valid_slices),
                       epochs = 60)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

deep_aug_model_600_005.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

deep_aug_model_600_005.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (188, 22, 1000)
Training label shape for 1 person: (188,)
Validation data shape for 1 person: (48, 22, 1000)
Validation label shape for 1 person: (48,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)
Training data shape with slices: (188, 22, 600)
Training label shape with slice: (188,)
Validation data shape with slices: (48, 22, 600)
Validation label shape with slice: (48,)
Train on 188 samples, validate on 48 samples
Epoch 1/60
188/188 [==============================] - 1s 4ms/sample - loss: 4.4071 - acc: 0.2713 - val_loss: 1.3895 - val_acc: 0.2500
Epoch 2/60
188/188 [==============================] - 0s 386us/sample - loss: 2.0831 - acc: 0.2660 - val_loss: 2.1183 - val_acc: 0.2708
Epoch 3/60
188/188 [==============================] - 0s 377us/sample - loss: 1.8098 - acc: 0.2500 - val_loss: 1.8270 - val_acc: 0.2292
Epoch 4/60
188/188 [==============================] - 0s 373us/sample - loss: 1.8765 - acc: 0.2340 - va

[2.503112441530077, 0.35214448]

In [30]:
person_num = 2
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))

deep_aug_model_600_005 = construct_augmented_deep_model(TIME_WINDOW)
deep_aug_model_600_005.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

deep_aug_model_600_005.fit(X_train_slices, y_train_slices,
                       validation_data = (X_valid_slices, y_valid_slices),
                       epochs = 60)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

deep_aug_model_600_005.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

deep_aug_model_600_005.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (188, 22, 1000)
Training label shape for 1 person: (188,)
Validation data shape for 1 person: (48, 22, 1000)
Validation label shape for 1 person: (48,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)
Training data shape with slices: (188, 22, 600)
Training label shape with slice: (188,)
Validation data shape with slices: (48, 22, 600)
Validation label shape with slice: (48,)
Train on 188 samples, validate on 48 samples
Epoch 1/60
188/188 [==============================] - 1s 4ms/sample - loss: 2.4297 - acc: 0.3138 - val_loss: 1.5118 - val_acc: 0.3125
Epoch 2/60
188/188 [==============================] - 0s 359us/sample - loss: 1.9522 - acc: 0.2606 - val_loss: 1.5750 - val_acc: 0.2083
Epoch 3/60
188/188 [==============================] - 0s 385us/sample - loss: 1.7718 - acc: 0.2606 - val_loss: 1.6682 - val_acc: 0.3125
Epoch 4/60
188/188 [==============================] - 0s 393us/sample - loss: 1.7338 - acc: 0.2766 - va

[3.318917728170046, 0.37246048]

In [31]:
person_num = 3
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))

deep_aug_model_600_005 = construct_augmented_deep_model(TIME_WINDOW)
deep_aug_model_600_005.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

deep_aug_model_600_005.fit(X_train_slices, y_train_slices,
                       validation_data = (X_valid_slices, y_valid_slices),
                       epochs = 60)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

deep_aug_model_600_005.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

deep_aug_model_600_005.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (187, 22, 1000)
Training label shape for 1 person: (187,)
Validation data shape for 1 person: (47, 22, 1000)
Validation label shape for 1 person: (47,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)
Training data shape with slices: (187, 22, 600)
Training label shape with slice: (187,)
Validation data shape with slices: (47, 22, 600)
Validation label shape with slice: (47,)
Train on 187 samples, validate on 47 samples
Epoch 1/60
187/187 [==============================] - 1s 5ms/sample - loss: 3.5911 - acc: 0.2299 - val_loss: 1.5121 - val_acc: 0.3191
Epoch 2/60
187/187 [==============================] - 0s 468us/sample - loss: 2.2870 - acc: 0.2193 - val_loss: 1.3597 - val_acc: 0.2553
Epoch 3/60
187/187 [==============================] - 0s 417us/sample - loss: 1.8868 - acc: 0.2941 - val_loss: 1.5079 - val_acc: 0.4255
Epoch 4/60
187/187 [==============================] - 0s 400us/sample - loss: 1.8241 - acc: 0.2513 - va

[2.268008071197613, 0.3476298]

In [32]:
person_num = 4
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))

deep_aug_model_600_005 = construct_augmented_deep_model(TIME_WINDOW)
deep_aug_model_600_005.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

deep_aug_model_600_005.fit(X_train_slices, y_train_slices,
                       validation_data = (X_valid_slices, y_valid_slices),
                       epochs = 60)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

deep_aug_model_600_005.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

deep_aug_model_600_005.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (188, 22, 1000)
Training label shape for 1 person: (188,)
Validation data shape for 1 person: (47, 22, 1000)
Validation label shape for 1 person: (47,)
Test data shape for 1 person: (47, 22, 1000)
Test label shape for 1 person: (47,)
Training data shape with slices: (188, 22, 600)
Training label shape with slice: (188,)
Validation data shape with slices: (47, 22, 600)
Validation label shape with slice: (47,)
Train on 188 samples, validate on 47 samples
Epoch 1/60
188/188 [==============================] - 1s 5ms/sample - loss: 3.8058 - acc: 0.2660 - val_loss: 1.3954 - val_acc: 0.3191
Epoch 2/60
188/188 [==============================] - 0s 453us/sample - loss: 1.9487 - acc: 0.2606 - val_loss: 1.3527 - val_acc: 0.3404
Epoch 3/60
188/188 [==============================] - 0s 407us/sample - loss: 1.8732 - acc: 0.2606 - val_loss: 1.2911 - val_acc: 0.4043
Epoch 4/60
188/188 [==============================] - 0s 408us/sample - loss: 1.8054 - acc: 0.2500 - va

[3.2393899372953325, 0.34537247]

In [34]:
person_num = 7
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))

deep_aug_model_600_005 = construct_augmented_deep_model(TIME_WINDOW)
deep_aug_model_600_005.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

deep_aug_model_600_005.fit(X_train_slices, y_train_slices,
                       validation_data = (X_valid_slices, y_valid_slices),
                       epochs = 60)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

deep_aug_model_600_005.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

deep_aug_model_600_005.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (185, 22, 1000)
Training label shape for 1 person: (185,)
Validation data shape for 1 person: (47, 22, 1000)
Validation label shape for 1 person: (47,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)
Training data shape with slices: (185, 22, 600)
Training label shape with slice: (185,)
Validation data shape with slices: (47, 22, 600)
Validation label shape with slice: (47,)
Train on 185 samples, validate on 47 samples
Epoch 1/60
185/185 [==============================] - 1s 7ms/sample - loss: 3.3258 - acc: 0.2757 - val_loss: 1.9008 - val_acc: 0.3191
Epoch 2/60
185/185 [==============================] - 0s 416us/sample - loss: 2.1526 - acc: 0.2486 - val_loss: 1.9341 - val_acc: 0.3191
Epoch 3/60
185/185 [==============================] - 0s 369us/sample - loss: 1.6887 - acc: 0.3297 - val_loss: 1.9784 - val_acc: 0.2128
Epoch 4/60
185/185 [==============================] - 0s 439us/sample - loss: 1.8072 - acc: 0.2811 - va

[2.612136176692713, 0.3363431]

In [33]:
person_num = 8
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))

deep_aug_model_600_005 = construct_augmented_deep_model(TIME_WINDOW)
deep_aug_model_600_005.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

deep_aug_model_600_005.fit(X_train_slices, y_train_slices,
                       validation_data = (X_valid_slices, y_valid_slices),
                       epochs = 60)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

deep_aug_model_600_005.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

deep_aug_model_600_005.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (184, 22, 1000)
Training label shape for 1 person: (184,)
Validation data shape for 1 person: (47, 22, 1000)
Validation label shape for 1 person: (47,)
Test data shape for 1 person: (47, 22, 1000)
Test label shape for 1 person: (47,)
Training data shape with slices: (184, 22, 600)
Training label shape with slice: (184,)
Validation data shape with slices: (47, 22, 600)
Validation label shape with slice: (47,)
Train on 184 samples, validate on 47 samples
Epoch 1/60
184/184 [==============================] - 1s 7ms/sample - loss: 2.9552 - acc: 0.2989 - val_loss: 1.8001 - val_acc: 0.2340
Epoch 2/60
184/184 [==============================] - 0s 371us/sample - loss: 1.9708 - acc: 0.2772 - val_loss: 1.4615 - val_acc: 0.2340
Epoch 3/60
184/184 [==============================] - 0s 377us/sample - loss: 1.5908 - acc: 0.2826 - val_loss: 1.2493 - val_acc: 0.3617
Epoch 4/60
184/184 [==============================] - 0s 375us/sample - loss: 1.4390 - acc: 0.3641 - va

[8.653981265849508, 0.27765238]